# **Bioinformatics Project - Computational Drug Discovery [Part 1] Download Bioactivity Data (Concised version)**

Chanin Nantasenamat

[*'Data Professor' YouTube channel*](http://youtube.com/dataprofessor)

In this Jupyter notebook, we will be building a real-life **data science project** that you can include in your **data science portfolio**. Particularly, we will be building a machine learning model using the ChEMBL bioactivity data.

In **Part 1**, we will be performing Data Collection and Pre-Processing from the ChEMBL Database.

Note for this Concised Version:
* Redundant code cells were deleted.
* Code cells for saving files to Google Drive has been deleted.

---

## **ChEMBL Database**

The [*ChEMBL Database*](https://www.ebi.ac.uk/chembl/) is a database that contains curated bioactivity data of more than 2 million compounds. It is compiled from more than 76,000 documents, 1.2 million assays and the data spans 13,000 targets and 1,800 cells and 33,000 indications.
[Data as of March 25, 2020; ChEMBL version 26].

## **Installing libraries**

Install the ChEMBL web service package so that we can retrieve bioactivity data from the ChEMBL Database.

In [1]:
! pip install chembl_webresource_client

     |████████████████████████████████| 55 kB 3.4 MB/s 
     |████████████████████████████████| 49 kB 6.4 MB/s 
     |████████████████████████████████| 636 kB 38.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Attempting uninstall: itsdangerous
    Found existing installation: itsdangerous 1.1.0
    Uninstalling itsdangerous-1.1.0:
      Successfully uninstalled itsdangerous-1.1.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
flask 1.1.4 requires itsdangerous<2.0,>=0.24, but you have itsdangerous 2.0.1 which is incompatible.


## **Importing libraries**

In [2]:
# Import necessary libraries
import pandas as pd
from chembl_webresource_client.new_client import new_client

## **Search for Target protein**

### **Target search for Acetylcholinesterase**

In [3]:
# Target search for coronavirus
target = new_client.target
target_query = target.search('acetylcholinesterase')
targets = pd.DataFrame.from_dict(target_query)
targets

cross_references  ...  tax_id
0   [{'xref_id': 'P22303', 'xref_name': None, 'xre...  ...    9606
1                                                  []  ...    9606
2                                                  []  ...    7227
3   [{'xref_id': 'P04058', 'xref_name': None, 'xre...  ...    7787
4   [{'xref_id': 'P21836', 'xref_name': None, 'xre...  ...   10090
5   [{'xref_id': 'P37136', 'xref_name': None, 'xre...  ...   10116
6   [{'xref_id': 'O42275', 'xref_name': None, 'xre...  ...    8005
7   [{'xref_id': 'P23795', 'xref_name': None, 'xre...  ...    9913
8                                                  []  ...    7165
9                                                  []  ...    7038
10                                                 []  ...    7539
11                                                 []  ...   94400
12                                                 []  ...    9031
13                                                 []  ...    7955
14                                                 []  ...    7159
15  [{'xref_id': 'Q95P20', 'xref_name': None, 'xre...  ...    7370
16                                                 []  ...   51655
17                                                 []  ...    7370
18                                                 []  ...  121226
19                                                 []  ...   69820
20                                                 []  ...  195883
21                                                 []  ...   13262
22                                                 []  ...    6239
23                                                 []  ...   10116

[24 rows x 9 columns]

### **Select and retrieve bioactivity data for *Human Acetylcholinesterase* (first entry)**

We will assign the fifth entry (which corresponds to the target protein, *Human Acetylcholinesterase*) to the ***selected_target*** variable 

In [4]:
selected_target = targets.target_chembl_id[0]
selected_target

'CHEMBL220'

Here, we will retrieve only bioactivity data for *Human Acetylcholinesterase* (CHEMBL220) that are reported as pChEMBL values.

In [5]:
activity = new_client.activity
res = activity.filter(target_chembl_id=selected_target).filter(standard_type="IC50")

In [6]:
df = pd.DataFrame.from_dict(res)

In [7]:
df

activity_comment  activity_id  ... upper_value  value
0                None        33969  ...        None   0.75
1                None        37563  ...        None    0.1
2                None        37565  ...        None   50.0
3                None        38902  ...        None    0.3
4                None        41170  ...        None    0.8
...               ...          ...  ...         ...    ...
7544             None     20703835  ...        None   0.74
7545             None     20703856  ...        None   0.51
7546             None     20708928  ...        None  125.0
7547             None     20708929  ...        None  100.0
7548             None     20708930  ...        None    0.1

[7549 rows x 45 columns]

Finally we will save the resulting bioactivity data to a CSV file **bioactivity_data.csv**.

In [8]:
df.to_csv('acetylcholinesterase_01_bioactivity_data_raw.csv', index=False)

## **Handling missing data**
If any compounds has missing value for the **standard_value** and **canonical_smiles** column then drop it.

In [9]:
df2 = df[df.standard_value.notna()]
df2 = df2[df.canonical_smiles.notna()]
df2

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


activity_comment  activity_id  ... upper_value  value
0                None        33969  ...        None   0.75
1                None        37563  ...        None    0.1
2                None        37565  ...        None   50.0
3                None        38902  ...        None    0.3
4                None        41170  ...        None    0.8
...               ...          ...  ...         ...    ...
7544             None     20703835  ...        None   0.74
7545             None     20703856  ...        None   0.51
7546             None     20708928  ...        None  125.0
7547             None     20708929  ...        None  100.0
7548             None     20708930  ...        None    0.1

[6340 rows x 45 columns]

In [10]:
len(df2.canonical_smiles.unique())

5103

In [11]:
df2_nr = df2.drop_duplicates(['canonical_smiles'])
df2_nr

activity_comment  activity_id  ... upper_value  value
0                None        33969  ...        None   0.75
1                None        37563  ...        None    0.1
2                None        37565  ...        None   50.0
3                None        38902  ...        None    0.3
4                None        41170  ...        None    0.8
...               ...          ...  ...         ...    ...
7543             None     20703806  ...        None  266.0
7544             None     20703835  ...        None   0.74
7545             None     20703856  ...        None   0.51
7546             None     20708928  ...        None  125.0
7547             None     20708929  ...        None  100.0

[5103 rows x 45 columns]

## **Data pre-processing of the bioactivity data**

### **Combine the 3 columns (molecule_chembl_id,canonical_smiles,standard_value) and bioactivity_class into a DataFrame**

In [12]:
selection = ['molecule_chembl_id','canonical_smiles','standard_value']
df3 = df2_nr[selection]
df3

molecule_chembl_id  ... standard_value
0          CHEMBL133897  ...          750.0
1          CHEMBL336398  ...          100.0
2          CHEMBL131588  ...        50000.0
3          CHEMBL130628  ...          300.0
4          CHEMBL130478  ...          800.0
...                 ...  ...            ...
7543      CHEMBL4645476  ...       266000.0
7544      CHEMBL4645659  ...          740.0
7545       CHEMBL513063  ...          510.0
7546      CHEMBL4640608  ...       125000.0
7547      CHEMBL4173961  ...       100000.0

[5103 rows x 3 columns]

Saves dataframe to CSV file

In [13]:
df3.to_csv('acetylcholinesterase_02_bioactivity_data_preprocessed.csv', index=False)

### **Labeling compounds as either being active, inactive or intermediate**
The bioactivity data is in the IC50 unit. Compounds having values of less than 1000 nM will be considered to be **active** while those greater than 10,000 nM will be considered to be **inactive**. As for those values in between 1,000 and 10,000 nM will be referred to as **intermediate**. 

In [14]:
df4 = pd.read_csv('acetylcholinesterase_02_bioactivity_data_preprocessed.csv')

In [15]:
bioactivity_threshold = []
for i in df4.standard_value:
  if float(i) >= 10000:
    bioactivity_threshold.append("inactive")
  elif float(i) <= 1000:
    bioactivity_threshold.append("active")
  else:
    bioactivity_threshold.append("intermediate")

In [16]:
bioactivity_class = pd.Series(bioactivity_threshold, name='class')
df5 = pd.concat([df4, bioactivity_class], axis=1)
df5

molecule_chembl_id  ...     class
0          CHEMBL133897  ...    active
1          CHEMBL336398  ...    active
2          CHEMBL131588  ...  inactive
3          CHEMBL130628  ...    active
4          CHEMBL130478  ...    active
...                 ...  ...       ...
5098      CHEMBL4645476  ...  inactive
5099      CHEMBL4645659  ...    active
5100       CHEMBL513063  ...    active
5101      CHEMBL4640608  ...  inactive
5102      CHEMBL4173961  ...  inactive

[5103 rows x 4 columns]

Saves dataframe to CSV file

In [17]:
df5.to_csv('acetylcholinesterase_03_bioactivity_data_curated.csv', index=False)

In [18]:
! zip acetylcholinesterase.zip *.csv

  adding: acetylcholinesterase_01_bioactivity_data_raw.csv (deflated 91%)
  adding: acetylcholinesterase_02_bioactivity_data_preprocessed.csv (deflated 81%)
  adding: acetylcholinesterase_03_bioactivity_data_curated.csv (deflated 82%)


In [19]:
! ls -l

total 5076
-rw-r--r-- 1 root root 3870914 Aug  2 11:11 acetylcholinesterase_01_bioactivity_data_raw.csv
-rw-r--r-- 1 root root  374458 Aug  2 11:11 acetylcholinesterase_02_bioactivity_data_preprocessed.csv
-rw-r--r-- 1 root root  421009 Aug  2 11:11 acetylcholinesterase_03_bioactivity_data_curated.csv
-rw-r--r-- 1 root root  516614 Aug  2 11:11 acetylcholinesterase.zip
drwxr-xr-x 1 root root    4096 Jul 16 13:20 sample_data


---